In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

## Illuminate a Sample with a LED Grid for FP in Blender
Importing a mesh and calling conveniently the functions below, you can obtain a Lo-Res image set as in **1. Forward Imaging Model** of fwdimaging Jupyter module.

It is suggested to export this notebook as Python script: **File**&rarr;**Download as**&rarr;**Python (.py)**.

See the example at the end of this notebook.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import bpy
import math
import os

In [ ]:
def set_up_sc(sc, **kwargs):
    """Set up some parameters of a scene, sc ('bpy.context.scene')."""
    wpx = kwargs.pop('wpx', 64)
    hpx = kwargs.pop('hpx', 64)
    antialias = kwargs.pop('antialias', '8')
    bw = kwargs.pop('bw', True)
    extension = kwargs.pop('extension', 'TIFF')
    sc.render.resolution_x = wpx
    sc.render.resolution_y = hpx
    sc.render.antialiasing_samples = antialias
    sc.render.use_overwrite = True
    sc.render.image_settings.file_format = extension
    if bw:
        sc.render.image_settings.color_mode = 'BW'
    else:
        sc.render.image_settings.color_mode = 'RGB'

In [ ]:
def delete_all_objects():
    """Delete all objects of the current blend file."""
    l_objects = list(bpy.data.objects)
    for o in l_objects:  # Select all objects.
        o.select = True
    bpy.ops.object.delete()  # Delete the selected objects.

In [ ]:
def num_str_zeros(num, n_digs, matlab=False):
    """Return a string that contains a sequence n-zeros followed by
    num ('int') as 'str', for example, num_str_zeros(89, 4) returns
    '0089'. From fwdimaging.ipynb Jupyter notebook."""
    if matlab:  # Begin the numeration with 1 in the filename.
        num += 1
    len_num = len(str(num))
    str_num = ''
    for i in range(n_digs - len_num):
        str_num += '0'
    str_num += str(num)
    return str_num

In [ ]:
def create_led_grid(sc, n=15, d=4, h=90, **kwargs):
    """Return the number of LEDs per row or column, the camera
    ('bpy.types.Camera') and the list with (x, y, z) coordinates
    ('tuple') of the all LED of the grid.
    
    n ('int') is the number of LEDs per row or per column in the grid
    LED, d ('int') is the distance in mm between LEDs and h ('int')
    is the distance between the sample and the grid.
    """
    radiusled = kwargs.pop('radiusled', 1.25)  # Radius in mm of the
                                               # LED.
    coordcam = kwargs.pop('coorcam', (0, 0, h - 1))
    radiusbase = kwargs.pop('radiusbase', (((n - 1) * d) / 2) * 1.5)
    # Create the base to put the object to be illuminated.
    bpy.ops.mesh.primitive_cylinder_add(location=(0, 0,0),
                                        radius=radiusbase,
                                        depth=0)
    # List of tuples with all coordinates of the LEDs of the grid.
    led_grid_coord = list()
    x_max = math.floor((n / 2)) * d
    x_min = -x_max
    y_max = x_max
    y_min = x_min
    led_grid_coord = list()
    l_range = list(range(x_min, x_max + 1, d))
    for i in l_range:
        for j in l_range:
            led_grid_coord.append((j, -i, h))
    len_gc = len(led_grid_coord)
    for i in range(len_gc):
        x_led, y_led, z_led = led_grid_coord[i]
        lamp_name = "lamp" + num_str_zeros(i, len(str(n ** 2)),
                                           matlab=True)
        # The lamp as a point.
        lamp_data = bpy.data.lamps.new(name=lamp_name, type='POINT')
        lamp_data.energy = 0.0 # The lamp if off.
        lamp_object = bpy.data.objects.new(name=lamp_name,
                                           object_data=lamp_data)
        sc.objects.link(lamp_object)
        lamp_object.location = (x_led, y_led, z_led)
        lamp_object.select = True
        sc.objects.active = lamp_object
        # Base model for the LED.
        bpy.ops.mesh.primitive_cylinder_add(location=(x_led,
                                                      y_led,
                                                      z_led),
                                            radius=radiusled,
                                            depth=0)
    # Create the camera.
    cam_name = "camgrid"
    cam_data = bpy.data.cameras.new(name=cam_name)
    cam_object = bpy.data.objects.new(name=cam_name,
                                      object_data=cam_data)
    scene.objects.link(cam_object)
    cam_object.location = coordcam
    sc.objects.active = cam_object
    # cam_object ('bpy.types.Camera') MUST BE RETURNED to render the
    # scene.
    return n, cam_object, led_grid_coord

In [ ]:
def take_photo(cam, sc, filename, outpath='img_lores/'):
    """Render a scene ('bpy.context.scene'), sc, with a camera
    ('bpy.types.Camera'), cam, and save the result in render_path
    ('str') as PNG file."""
    sc.camera = cam
    sc.render.filepath = outpath + filename
    bpy.ops.render.render(write_still=True)

In [ ]:
def illuminate_step_by_step(n, cam, sc, **kwargs):
    """Turn on a lamp at time, take a photo of the mesh and save it.
    """
    prefix = kwargs.pop('prefix', 'lores_')
    l_lamps = get_lamps()
    len_ll = len(l_lamps)
    for i in range(len_ll):
        l_lamps[i].data.energy = 2.0
        take_photo(cam, sc, prefix + num_str_zeros(i,
                                                   len(str(n ** 2)),
                                                   matlab=True))
        l_lamps[i].data.energy = 0.0

In [ ]:
def get_lamps(prefix='lamp'):
    """Return a list ('list') with all lamps ('bpy.types.Lamp') of the
    blend file."""
    l_lamps = list()
    for o in bpy.data.objects:
        if prefix in o.name:
            l_lamps.append(o)
    return l_lamps

### Example

Uncomment the first line below and **execute once** before import your mesh.

In [ ]:
# delete_all_objects()
# UNCOMMENT ALL BELOW to render the scene.
# scene = bpy.context.scene
# set_up_sc(scene)
# n, cam, _ = create_led_grid(scene)
# illuminate_step_by_step(n, cam, scene)